In [34]:
import os
import re
import pandas as pd
import yaml

import wombat.bsub as bsub

In [35]:
# !pip install git+https://github.com/estorrs/wombat.git

In [36]:
# !pip uninstall wombat -y

###### grab data from cptac bammap

In [37]:
# df = pd.read_csv('/home/estorrs/CPTAC3.catalog/BamMap/storage1.BamMap.dat', sep='\t', index_col=0)
# df = df[df['experimental_strategy']=='WXS']
# df = df[df['sample_type']=='blood_normal']
# df = df[df['reference']=='hg38']
# df

In [81]:
df = pd.read_csv('/diskmnt/Projects/Users/estorrs/pecgs-cnv/data/pecgs_analysis_summary.txt', sep='\t', header=None)
df = df.iloc[:, [0, 1, 2, 3]]
df.columns = ['case_id', 'step', 'filetype', 'filepath']
df = df[df['step']=='align_tumor_wxs']
df

,case_id,step,filetype,filepath
0,PE001C1,align_tumor_wxs,tumor_wxs_output_bam,/storage1/fs1/dinglab/Active/Projects/PECGS/PE...
2,PE002C1,align_tumor_wxs,tumor_wxs_output_bam,/storage1/fs1/dinglab/Active/Projects/PECGS/PE...
4,PE0038U1,align_tumor_wxs,tumor_wxs_output_bam,/storage1/fs1/dinglab/Active/Projects/PECGS/PE...
6,PE003C1,align_tumor_wxs,tumor_wxs_output_bam,/storage1/fs1/dinglab/Active/Projects/PECGS/PE...
8,PE0043A,align_tumor_wxs,tumor_wxs_output_bam,/storage1/fs1/dinglab/Active/Projects/PECGS/PE...
10,PE005A1,align_tumor_wxs,tumor_wxs_output_bam,/storage1/fs1/dinglab/Active/Projects/PECGS/PE...
12,PE0078U1,align_tumor_wxs,tumor_wxs_output_bam,/storage1/fs1/dinglab/Active/Projects/PECGS/PE...
14,PEx011U1,align_tumor_wxs,tumor_wxs_output_bam,/storage1/fs1/dinglab/Active/Projects/PECGS/PE...
16,PEx012U1,align_tumor_wxs,tumor_wxs_output_bam,/storage1/fs1/dinglab/Active/Projects/PECGS/PE...


In [82]:
df = df.iloc[[i for i in range(df.shape[0]) if i!=2]]
df

,case_id,step,filetype,filepath
0,PE001C1,align_tumor_wxs,tumor_wxs_output_bam,/storage1/fs1/dinglab/Active/Projects/PECGS/PE...
2,PE002C1,align_tumor_wxs,tumor_wxs_output_bam,/storage1/fs1/dinglab/Active/Projects/PECGS/PE...
6,PE003C1,align_tumor_wxs,tumor_wxs_output_bam,/storage1/fs1/dinglab/Active/Projects/PECGS/PE...
8,PE0043A,align_tumor_wxs,tumor_wxs_output_bam,/storage1/fs1/dinglab/Active/Projects/PECGS/PE...
10,PE005A1,align_tumor_wxs,tumor_wxs_output_bam,/storage1/fs1/dinglab/Active/Projects/PECGS/PE...
12,PE0078U1,align_tumor_wxs,tumor_wxs_output_bam,/storage1/fs1/dinglab/Active/Projects/PECGS/PE...
14,PEx011U1,align_tumor_wxs,tumor_wxs_output_bam,/storage1/fs1/dinglab/Active/Projects/PECGS/PE...
16,PEx012U1,align_tumor_wxs,tumor_wxs_output_bam,/storage1/fs1/dinglab/Active/Projects/PECGS/PE...


In [83]:
df['filepath'][0]

'/storage1/fs1/dinglab/Active/Projects/PECGS/PECGS_analysis/pecgs_batch1/runs/PE001C1_1c32a190-745c-421b-9f2b-eee3ea6e9e81/cromwell-workdir/cromwell-executions/pecgs_TN_wxs_fq.cwl/877c6c49-90a7-40af-8330-b81027660cdc/call-align_tumor_wxs/execution/output.bam'

In [84]:
# GENOME=/storage1/fs1/dinglab/Active/Projects/austins2/references/GRCh38.d1.vd1/GRCh38.d1.vd1.fa
# GENOME_DICT=/storage1/fs1/dinglab/Active/Projects/austins2/references/GRCh38.d1.vd1/GRCh38.d1.vd1.dict


# # Target
# TargetIntervalList=/code/db/IDT_xGen_Exome_Research_Panel_v1.hg38.removed_alt_chr.bed.target.preprocessed.exome.interval_list


# # SNP database
# COMMON_BIALLELIC=/code/db/snp_database/af-only-gnomad.hg38.common_biallelic.chr1-22XY.vcf


# # protein coding genes
# ProteinCodingGene=/code/db/remaking_protein-coding/gencode.v34.annotation.gene_filterd.need_gene_symbol.no_sym.filtered_to_hgnc_protein-coding_genes.bed


In [85]:
genome = '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cnv/references/GRCh38.d1.vd1/GRCh38.d1.vd1.fa'
genome_dict = '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cnv/references/GRCh38.d1.vd1/GRCh38.d1.vd1.dict'
target_interval_list = '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cnv/db/IDT_xGen_Exome_Research_Panel_v1.hg38.removed_alt_chr.autosome_only.bed.preprocessed.exome.interval_list'
common_biallelic = '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cnv/db/snp_database/af-only-gnomad.hg38.common_biallelic.chr1-22XY.vcf'
protein_coding_gene = '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cnv/db/gencode.v34.annotation.gene_filtered.protein_coding.ensembl_ID_no_version.protein-coding_hgnc_filtered.duplicates_removed.ensembl_ID_removed.txt'
cytoband = '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cnv/db/cytoBand.txt'







In [86]:
pon_out_dir = '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cnv/pon/pecgs_pon_v1'

In [87]:
args = bsub.DEFAULT_ARGS
args['n_concurrent'] = 25
args['mem'], args['max_mem'] = 28, 28
args['docker'] = 'estorrs/pecgs_cnv:0.0.2'
args['group_name'] = 'pecgs-25'
args

{'mem': 28,
 'n_processes': 1,
 'max_mem': 28,
 'docker': 'estorrs/pecgs_cnv:0.0.2',
 'queue': 'dinglab',
 'gpu_model': 'TeslaV100_SXM2_32GB',
 'gpu_mem': '30',
 'gpu_num': 1,
 'use_gpu': False,
 'group': 'compute-dinglab',
 'group_name': 'pecgs-25',
 'n_concurrent': 25,
 'interactive': False,
 'username': 'estorrs'}

###### step 1: precall

In [88]:
# bash /storage1/fs1/dinglab/Active/Projects/austins2/tools/GATK4SCNA/gatk_somatic.cnv.mgi.sh -p precall -t /storage1/fs1/dinglab/Active/Projects/austins2/HTAN-BRCA/CNV/gatk4scna/bam.list -M /storage1/fs1/dinglab/Active/Projects/austins2/HTAN-BRCA/CNV/gatk4scna/normalBam.list -o /storage1/fs1/dinglab/Active/Projects/austins2/HTAN-BRCA/CNV/gatk4scna -c /storage1/fs1/dinglab/Active/Projects/austins2/tools/GATK4SCNA/config/config.gatk4scna.mgi.ini

# sh $submitJob 4 1 gatk4cn.s1.${name} "bash $scriptDir/src/1.run.gatk4scna.collectReadCounts.Normal.sh -C ${config} -N ${caseID} -T Normal -B ${normalBam} -O ${outdir}"




In [89]:
# ${JAVA} -Dsamjdk.use_async_io_read_samtools=false \
#       -Dsamjdk.use_async_io_write_samtools=true \
#       -Dsamjdk.use_async_io_write_tribble=false \
#       -Dsamjdk.compression_level=2 \
#       -Xmx4g -jar ${GATK} CollectReadCounts \
#       -I ${BAM} \
#       -L ${TargetIntervalList} \
#       --interval-merging-rule OVERLAPPING_ONLY \
#       -O ${OUT}/${NAME}.N.counts.hdf5

In [90]:
def get_precall_command(sample, normal_fp, out_dir):
    pieces = [
        'gatk CollectReadCounts',
        '-I', normal_fp,
        '-L', target_interval_list,
        '-interval-merging-rule OVERLAPPING_ONLY',
        f'-O {out_dir}/{sample}.N.counts.hdf5'
    ]
    cmd = ' '.join(pieces)
    return cmd


In [91]:
precall_out_dir = os.path.join(pon_out_dir, 'precall')
cmds = []
for i, row in df.iterrows():
    cmd = f'export PATH=/miniconda/envs/cnv/bin:$PATH && ' + get_precall_command(row['case_id'], row['filepath'], precall_out_dir)
    cmds.append(cmd)
cmds[:2]

['export PATH=/miniconda/envs/cnv/bin:$PATH && gatk CollectReadCounts -I /storage1/fs1/dinglab/Active/Projects/PECGS/PECGS_analysis/pecgs_batch1/runs/PE001C1_1c32a190-745c-421b-9f2b-eee3ea6e9e81/cromwell-workdir/cromwell-executions/pecgs_TN_wxs_fq.cwl/877c6c49-90a7-40af-8330-b81027660cdc/call-align_tumor_wxs/execution/output.bam -L /storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cnv/db/IDT_xGen_Exome_Research_Panel_v1.hg38.removed_alt_chr.autosome_only.bed.preprocessed.exome.interval_list -interval-merging-rule OVERLAPPING_ONLY -O /storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cnv/pon/pecgs_pon_v1/precall/PE001C1.N.counts.hdf5',
 'export PATH=/miniconda/envs/cnv/bin:$PATH && gatk CollectReadCounts -I /storage1/fs1/dinglab/Active/Projects/PECGS/PECGS_analysis/pecgs_batch1/runs/PE002C1_dfd8fa5c-873e-4ce8-99eb-c8e361104428/cromwell-workdir/cromwell-executions/pecgs_TN_wxs_fq.cwl/dce57945-2818-45f0-8de7-797a8f2d4dab/call-align_tumor_wxs/execution/output.bam -L 

In [92]:
job_names = [f'{s}_precall' for s in df['case_id']]
volumes = ['/storage1/fs1/m.wyczalkowski', '/storage1/fs1/dinglab', '/scratch1/fs1/dinglab']
bsub_commands = bsub.batch_bsub_commands(cmds, job_names, os.path.join(precall_out_dir, 'logs'), args, volumes=volumes)
bsub_commands[:3]

['mkdir -p /storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cnv/pon/pecgs_pon_v1/precall/logs',
 'export LSF_DOCKER_VOLUMES="/storage1/fs1/m.wyczalkowski:/storage1/fs1/m.wyczalkowski /storage1/fs1/dinglab:/storage1/fs1/dinglab /scratch1/fs1/dinglab:/scratch1/fs1/dinglab /storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cnv/pon/pecgs_pon_v1/precall/logs:/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cnv/pon/pecgs_pon_v1/precall/logs"',
 'bgadd -L 25 /estorrs/pecgs-25']

In [93]:
bsub.write_command_file(bsub_commands, '../tests/1.precall.sh')

In [94]:
bsub.write_command_file(bsub_commands[:3], '../tests/1.precall_small.sh')

###### step 2: pool normal

In [95]:
# sh $submitJob 8 1 gatk4cn.s2.pon "bash $scriptDir/src/2.run.gatk4scna.createPON.sh -C ${config} -D ${hdf5Dir}"

In [96]:
# all_normal_hdf5=`ls ${DIR}/*.N.counts.hdf5 | while read file; do printf -- "-I $file "; done`
# ${JAVA} -Dsamjdk.use_async_io_read_samtools=false \
#       -Dsamjdk.use_async_io_write_samtools=true \
#       -Dsamjdk.use_async_io_write_tribble=false \
#       -Dsamjdk.compression_level=2 \
#       -Xmx8g -jar ${GATK} CreateReadCountPanelOfNormals \
#       --minimum-interval-median-percentile 5.0 \
#       -O ${DIR}/gatk4scnaPON.Normal.hdf5 \
#       $all_normal_hdf5

In [97]:
def get_pool_command(sample, fps, out_dir):
    pieces = [
        'gatk CreateReadCountPanelOfNormals',
        '--minimum-interval-median-percentile 5.0',
        f'-O {out_dir}/gatk4scnaPON.Normal.hdf5'
    ]
    pieces += [f'-I {fp}' for fp in fps]
    
    cmd = ' '.join(pieces)
    return cmd


In [98]:
pool_out_dir = os.path.join(pon_out_dir, 'create_pon')

In [99]:
fps = [f'{precall_out_dir}/{s}.N.counts.hdf5' for s in df['case_id'].to_list()]
fps

['/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cnv/pon/pecgs_pon_v1/precall/PE001C1.N.counts.hdf5',
 '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cnv/pon/pecgs_pon_v1/precall/PE002C1.N.counts.hdf5',
 '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cnv/pon/pecgs_pon_v1/precall/PE003C1.N.counts.hdf5',
 '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cnv/pon/pecgs_pon_v1/precall/PE0043A.N.counts.hdf5',
 '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cnv/pon/pecgs_pon_v1/precall/PE005A1.N.counts.hdf5',
 '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cnv/pon/pecgs_pon_v1/precall/PE0078U1.N.counts.hdf5',
 '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cnv/pon/pecgs_pon_v1/precall/PEx011U1.N.counts.hdf5',
 '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cnv/pon/pecgs_pon_v1/precall/PEx012U1.N.counts.hdf5']

In [100]:
cmd = f'export PATH=/miniconda/envs/cnv/bin:$PATH && ' + get_pool_command(df['case_id'], fps, pool_out_dir)
cmd

'export PATH=/miniconda/envs/cnv/bin:$PATH && gatk CreateReadCountPanelOfNormals --minimum-interval-median-percentile 5.0 -O /storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cnv/pon/pecgs_pon_v1/create_pon/gatk4scnaPON.Normal.hdf5 -I /storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cnv/pon/pecgs_pon_v1/precall/PE001C1.N.counts.hdf5 -I /storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cnv/pon/pecgs_pon_v1/precall/PE002C1.N.counts.hdf5 -I /storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cnv/pon/pecgs_pon_v1/precall/PE003C1.N.counts.hdf5 -I /storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cnv/pon/pecgs_pon_v1/precall/PE0043A.N.counts.hdf5 -I /storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cnv/pon/pecgs_pon_v1/precall/PE005A1.N.counts.hdf5 -I /storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cnv/pon/pecgs_pon_v1/precall/PE0078U1.N.counts.hdf5 -I /storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_reso

In [101]:
job_names = [f'create_pon']
volumes = ['/storage1/fs1/m.wyczalkowski', '/storage1/fs1/dinglab', '/scratch1/fs1/dinglab']
bsub_commands = bsub.batch_bsub_commands([cmd], ['create_pon'], pool_out_dir, args, volumes=volumes)
bsub_commands

['mkdir -p /storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cnv/pon/pecgs_pon_v1/create_pon',
 'export LSF_DOCKER_VOLUMES="/storage1/fs1/m.wyczalkowski:/storage1/fs1/m.wyczalkowski /storage1/fs1/dinglab:/storage1/fs1/dinglab /scratch1/fs1/dinglab:/scratch1/fs1/dinglab /storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cnv/pon/pecgs_pon_v1/create_pon:/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cnv/pon/pecgs_pon_v1/create_pon"',
 'bgadd -L 25 /estorrs/pecgs-25',
 'export PATH="/opt/java/openjdk/bin:$PATH"',
 "bsub -R 'select[mem>28GB] rusage[mem=28GB] span[hosts=1]' -M 28GB -n 1 -q dinglab -G compute-dinglab -a 'docker(estorrs/pecgs_cnv:0.0.2)' -g /estorrs/pecgs-25 -J create_pon -oo /storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cnv/pon/pecgs_pon_v1/create_pon/create_pon.txt 'export PATH=/miniconda/envs/cnv/bin:$PATH && gatk CreateReadCountPanelOfNormals --minimum-interval-median-percentile 5.0 -O /storage1/fs1/dinglab/Active/Proje

In [102]:
bsub.write_command_file(bsub_commands, '../tests/2.create_pon.sh')